In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

## Challenge

Determind the five most commone journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

In [73]:
# upload the data file that contains the dirty data
dirty = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv',encoding='"ISO-8859-1"')
dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 41.6+ KB


In [74]:
df = dirty.copy()
df = df.rename(columns={'PMID/PMCID':'ID', 'Publisher':'Publisher', 'Journal title':'Journal', 'Article title':'Article',
                        'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'})
df.head()

,ID,Publisher,Journal,Article,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
ID           1928 non-null object
Publisher    2127 non-null object
Journal      2126 non-null object
Article      2127 non-null object
Cost         2127 non-null object
dtypes: object(5)
memory usage: 41.6+ KB


In [76]:
# removing null values in Journal column
df=df[df['Journal'].notnull()][['ID', 'Publisher', 'Journal', 'Article', 'Cost']]

In [77]:
# Look at unique journal names
df['Journal'].value_counts()

PLoS One                                                    92
PLoS ONE                                                    62
Journal of Biological Chemistry                             48
Nucleic Acids Research                                      21
Proceedings of the National Academy of Sciences             19
PLoS Neglected Tropical Diseases                            18
Human Molecular Genetics                                    18
Nature Communications                                       17
PLoS Pathogens                                              15
Neuroimage                                                  15
PLoS Genetics                                               15
NeuroImage                                                  14
PLOS ONE                                                    14
BMC Public Health                                           14
Brain                                                       14
Movement Disorders                                     

## Cleaning dirty Journal column

1. Strip Whitespace
2. Convert to upper case
3. Remove misc. characters.
4. Spell Check

In [78]:
# Strip whitespace
df['clean_journal'] = df['Journal'].str.strip()

# convert to upper case
df['clean_journal'] = df['Journal'].str.upper()

df['clean_journal'].value_counts()

PLOS ONE                                            190
JOURNAL OF BIOLOGICAL CHEMISTRY                      53
NEUROIMAGE                                           29
PLOS PATHOGENS                                       24
PLOS GENETICS                                        24
NUCLEIC ACIDS RESEARCH                               23
PLOS NEGLECTED TROPICAL DISEASES                     20
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES      20
NATURE COMMUNICATIONS                                19
HUMAN MOLECULAR GENETICS                             19
BMC PUBLIC HEALTH                                    14
BRAIN                                                14
MOVEMENT DISORDERS                                   13
JOURNAL OF NEUROSCIENCE                              12
DEVELOPMENTAL CELL                                   12
BIOCHEMICAL JOURNAL                                  12
JOURNAL OF GENERAL VIROLOGY                          11
CURRENT BIOLOGY                                 

From this it is clear that some of the journals, for example, "PLOS ONE", have been cleaned.

In [79]:
def is_a_string(x):
    return not str(x).isalnum()

#print(df.clean_journal.apply(is_a_string))
print('Filtering the whole series:')
print(list(filter(lambda x: str(x).isalnum(), df.clean_journal)))

print('\nApplying filter() to each value in the series:')
print(df.clean_journal.apply(lambda x: ''.join(list(filter(str.isalnum, str(x))))))

Filtering the whole series:
['BIOMACROMOLECULES', 'BIOCHEMISTRY', 'GASTROENTEROLOGY', 'CHEST', 'EMOTION', 'MBIO', 'BLOOD', 'BLOOD', 'BLOOD', 'BLOOD', 'BLOOD', 'BLOOD', 'BLOOD', 'EVODEVO', 'TRIALS', 'TRIALS', 'TRIALS', 'TRIALS', 'BJOPHTHALMOL', 'BMJ', 'BMJ', 'BMJ', 'BMJ', 'GUT', 'BMJ', 'GUT', 'GUT', 'HEART', 'THORAX', 'THORAX', 'BMJ', 'BMJ', 'BMJ', 'BMJ', 'BMJ', 'MBIO', 'PARASITOLOGY', 'PARASITOLOGY', 'PARASITOLOGY', 'PARASITOLOGY', 'PARASITOLOGY', 'PARASITOLOGY', 'RNA', 'RNA', 'DEVELOPMENT', 'DEVELOPMENT', 'DEVELOPMENT', 'DEVELOPMENT', 'DEVELOPMENT', 'DEVELOPMENT', 'DEVELOPMENT', 'GENETICS', 'PNAS', 'NEUROIMAGE', 'NEUROIMAGE', 'ATHEROSCLEROSIS', 'ATHEROSCLEROSIS', 'BIOCHIMIE', 'BIOMATERIALS', 'BIOMATERIALS', 'BIOSYSTEMS', 'BONE', 'BONE', 'BONE', 'CELL', 'CELL', 'CELL', 'CELL', 'CELL', 'COGNITION', 'CORTEX', 'CORTEX', 'CORTEX', 'CORTEX', 'CORTEX', 'ELSEVIER', 'ELSEVIER', 'ELSEVIER', 'ELSEVIER', 'ELSEVIER', 'ELSEVIER', 'ELSEVIER', 'ENDEAVOUR', 'ENDEAVOUR', 'GASTROENTEROLOGY', 'GENE', 'GE

In [80]:
df['clean_journal'] = df['clean_journal'].str.replace('&', 'AND')
df['clean_journal'] = df['clean_journal'].str.replace('THE ', '')
df['clean_journal'] = df['clean_journal'].str.replace('.', '')
df['clean_journal'] = df['clean_journal'].str.replace('OF ', '')
df['clean_journal'] = df['clean_journal'].str.replace('FOR ', '')
df['clean_journal'] = df['clean_journal'].str.replace(' IN ', ' ')
df['clean_journal'] = df['clean_journal'].str.replace(':', '')
df['clean_journal'] = df['clean_journal'].str.replace(',', '')
df['clean_journal'] = df['clean_journal'].str.replace(' - ', ' ')
df['clean_journal'] = df['clean_journal'].str.replace('  ', ' ')
df['clean_journal'] = df['clean_journal'].str.strip()

In [81]:
df['clean_journal'].value_counts()

PLOS ONE                                    191
JOURNAL BIOLOGICAL CHEMISTRY                 61
NEUROIMAGE                                   29
NUCLEIC ACIDS RESEARCH                       26
PLOS GENETICS                                24
PLOS PATHOGENS                               24
PROCEEDINGS NATIONAL ACADEMY SCIENCES        22
PLOS NEGLECTED TROPICAL DISEASES             20
NATURE COMMUNICATIONS                        19
HUMAN MOLECULAR GENETICS                     19
MOVEMENT DISORDERS                           15
JOURNAL NEUROSCIENCE                         15
BMC PUBLIC HEALTH                            15
BRAIN                                        14
BIOCHEMICAL JOURNAL                          14
DEVELOPMENTAL CELL                           12
FASEB JOURNAL                                11
JOURNAL GENERAL VIROLOGY                     11
CURRENT BIOLOGY                              11
PLOS COMPUTATIONAL BIOLOGY                   10
BMJ                                     

In [82]:
df['clean_journal'] = df['clean_journal'].str.replace('AMERICAL','AMERICAN')
df['clean_journal'] = df['clean_journal'].str.replace('ANTIMICOBIAL','ANTIMICROBIAL')
df['clean_journal'] = df['clean_journal'].str.replace('AGFENTS','AGENTS')
df['clean_journal'] = df['clean_journal'].str.replace('BEHAVIOUR', 'BEHAVIOR')
df['clean_journal'] = df['clean_journal'].str.replace('BEHAVIOURAL', 'BEHAVIORAL')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNALS','JOURNAL')
df['clean_journal'] = df['clean_journal'].str.replace('JNL', 'JOURNAL')
df['clean_journal'] = df['clean_journal'].str.replace(' J ',' JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('BIOOHYSICA', 'BIOPHYSICA')
df['clean_journal'] = df['clean_journal'].str.replace('BIOLOGICIAL', 'BIOLOGICAL')
df['clean_journal'] = df['clean_journal'].str.replace('BMS ','BMC ')
df['clean_journal'] = df['clean_journal'].str.replace('BRT', 'BRITISH')
df['clean_journal'] = df['clean_journal'].str.replace('CLINICLA', 'CLINICAL')
df['clean_journal'] = df['clean_journal'].str.replace('CURR BIOL', 'CURRENT BIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('EPIGENTICS', 'EPIGENETICS')
df['clean_journal'] = df['clean_journal'].str.replace('PSYCHIATTY', 'PSYCHIATRY')
df['clean_journal'] = df['clean_journal'].str.replace('EPIDEMOLOGY', 'EPIDEMIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('HEPTOLOGY', 'HEPATOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('HUM RESOUR HEALTH', 'HUMAN RESOURCES HEALTH')
df['clean_journal'] = df['clean_journal'].str.replace('IMMNUNOBIOLOGY', 'IMMUNOBIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('INT', 'INTERNATIONAL')
df['clean_journal'] = df['clean_journal'].str.replace('INYERNATIONAL', 'INTERNATIONAL')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMS (JAIDS)', 
                                                      'JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNAL AIDS', 'JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNAL AUTISM AND DEVELOPMENT DISORDERS', 
                                                      'JOURNAL AUTISM AND DEVELOPMENTAL DISORDERS')
df['clean_journal'] = df['clean_journal'].str.replace('BIOL CHEM','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('BIOL CHEMISTRY','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('J BIOL CHEM','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('J BIOL CHEMISTRY','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNAL VISUALIZED EXPERIEMENTS', 'JOURNAL VISUALIZED EXPERIMENTS')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNAL VISULAIZED EXPERMIMENTS', 'JOURNAL VISUALIZED EXPERIMENTS')
df['clean_journal'] = df['clean_journal'].str.replace('MARTEN CHILD NUTR', 'MATERNAL AND CHILD NUTRITION')
df['clean_journal'] = df['clean_journal'].str.replace('MICROBES INFECT', 'MICROBES AND INFECTION')
df['clean_journal'] = df['clean_journal'].str.replace('MOL BIO', 'MOLECULAR BIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('NEPHROLOGY DIALYSIS TRANSPLANTATION', 
                                                      'NEPHROLOGY DIALYSIS AND TRANSPLANTATION')
df['clean_journal'] = df['clean_journal'].str.replace('NEUROLMAGE', 'NEUROIMAGE')
df['clean_journal'] = df['clean_journal'].str.replace('NUCLEIC ACID RESEARCH', 'NUCLEIC ACIDS RESEARCH')
df['clean_journal'] = df['clean_journal'].str.replace('PARASIT VECTORS', 'PARASITES AND VECTORS')
df['clean_journal'] = df['clean_journal'].str.replace('PEDIATR', 'PEDIATRIC')
df['clean_journal'] = df['clean_journal'].str.replace('PFLUGERS ARCHIV', 'PFLUGERS ARCHIVE')
df['clean_journal'] = df['clean_journal'].str.replace('PLOSONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PLOS  ONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PUBLIC LIBRARY SCIENCE ONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PUBLIC LIBRARY SCIENCE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PLOS 1','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('SEX TRANSM INFECT', 'SEXUALLY TRANSMITTED INFECTIONS')
df['clean_journal'] = df['clean_journal'].str.replace('SOCIAL PSYCHIATRY AND PSYCHIATRIC EPIDEMIOL', 
                                                      'SOCIAL PSYCHIATRY AND PSYCHIATRIC EPIDEMIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('TRENDS NEUROSCIENCE', 'TRENDS NEUROSCIENCES')

In [83]:
df['clean_journal'].value_counts()

PLOS ONE                                            209
JOURNAL BIOLOGICAL CHEMISTRY                         65
NEUROIMAGE                                           31
NUCLEIC ACIDS RESEARCH                               29
PLOS GENETICS                                        24
PLOS PATHOGENS                                       24
PROCEEDINGS NATIONAL ACADEMY SCIENCES                22
PLOS NEGLECTED TROPICAL DISEASES                     20
HUMAN MOLECULAR GENETICS                             19
NATURE COMMUNICATIONS                                19
MOVEMENT DISORDERS                                   15
BMC PUBLIC HEALTH                                    15
BIOCHEMICAL JOURNAL                                  15
JOURNAL NEUROSCIENCE                                 15
BRAIN                                                14
DEVELOPMENTAL CELL                                   12
CURRENT BIOLOGY                                      12
JOURNAL GENERAL VIROLOGY                        

In [84]:
#ACTA journals from International Union of Crystallography 
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA D' if 'ACTA CRYSTALLOGRAPHICA SECTION D' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA D' if 'ACTA CRYSTALLOGRAPHY D' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA F' if 'ACTA CRYSTALLOGRAPHICA SECTION F STRUCTURAL BIOLOGY AND CRYSTALLIZATION COMMUNICATIONS' in x else x)
#ACTA Neurpatholica
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA NEUROPATHOLOGICA' if 'ACTA NEUROPATHOL' in x else x)
#BIOLOGY OPEN
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOLOGY OPEN' if 'BIOL OPEN' in x else x)
#AIDS Journal by Wolters Kluwer
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AIDS JOURNAL' if 'AIDS' == x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AIDS JOURNAL' if 'AIDS UK' == x else x)
#CHILD CARE HEATH AND DEVELOPMENT
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CHILD CARE HEATH AND DEVELOPMENT' \
                                                if 'CHILD CARE HEALTH DEVELOPMENT' in x else x)
#BIOCHIMICA ET BIOOHYSICA ACTA MOLECULAR BASIS DISEASE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOCHIMICA ET BIOPHYSICA ACTA MOLECULAR BASIS DISEASE' \
                                                if 'BBA ' in x else x)
#CLINICAL INFECTIOUS DISEASES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CLINICAL INFECTIOUS DISEASES' \
                                                if 'CLINICLA INFECTIOUS DISEASE' in x else x)
#JOURNAL INFECTIOUS DISEASES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL INFECTIOUS DISEASES' \
                                                if 'J INFECT DIS' in x else x)
#EMBO JOURNAL
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'EMBO JOURNAL' if 'EMBO' == x else x)
#EUROPEAN JOURNAL IMMUNOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'EUROPEAN JOURNAL IMMUNOLOGY' if 'EUR JOURNAL IMMUNOL' in x else x)
#FEBS JOURNAL
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'FEBS JOURNAL' if 'FEBS J' == x else x)
#JOURNAL CLINICAL VIROLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CLINICAL VIROLOGY' if 'VIROLOGY' == x else x)
#HUMAN MUTATION
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'HUMAN MUTATION' if 'HUMAN MUTATION' in x else x)
#JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES'\
                                                if 'ACQUIRED IMMUNE DEFICIENCY' in x else x)
#JOURNAL IMMUNOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL IMMUNOLOGY' if 'J IMMUNOL' in x else x)
#JOURNAL CELL SCIENCE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CELL SCIENCE' if 'J CELL SCI' in x else x)
#JOURNAL CLINICAL MICROBIOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CLINICAL MICROBIOLOGY' if 'J CLIN MICROBIOL' in x else x)
#JOURNAL MEDICINAL CHEMISTRY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MEDICINAL CHEMISTRY' if 'J MED CHEM' == x else x)
#JOURNAL MECHANISMS AGEING AND DEVELOPMENT
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MECHANISMS AGEING AND DEVELOPMENT' \
                                                if 'MECHANISMS AGEING AND DEVELOPMENT' in x else x)
#JOURNAL MICROBIOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MICROBIOLOGY' if 'MICROBIOLOGY' == x else x)
#PARASITES AND VECTORS
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PARASITES AND VECTORS' if 'PARASIT VECTORS' == x else x)
#PROCEEDINGS NATIONAL ACADEMY SCIENCES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' if 'PNAS' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' if 'PROCEDDINGS' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROC NATL ACAD SCI U S A' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROCEEDINGS NATIONAL ACADEMY SCIENCES UNITED STATES AMERICA' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROCEEDINGS NATIONAL ACADEMY SCIENCES USA' in x else x)
#Public Library of Science
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PLOS ONE' if 'PLOS' == x else x)

In [85]:
df['clean_journal'] = df['clean_journal'].str.strip()
df['clean_journal'].value_counts()

PLOS ONE                                                                        213
JOURNAL BIOLOGICAL CHEMISTRY                                                     65
PROCEEDINGS NATIONAL ACADEMY SCIENCES                                            37
NEUROIMAGE                                                                       31
NUCLEIC ACIDS RESEARCH                                                           29
PLOS GENETICS                                                                    24
PLOS PATHOGENS                                                                   24
PLOS NEGLECTED TROPICAL DISEASES                                                 20
HUMAN MOLECULAR GENETICS                                                         19
NATURE COMMUNICATIONS                                                            19
BMC PUBLIC HEALTH                                                                15
JOURNAL NEUROSCIENCE                                                        

In [86]:
df.groupby('clean_journal').count()

,ID,Publisher,Journal,Article,Cost
clean_journal,,,,,
ACADEMY NUTRITION AND DIETETICS,1,1,1,1,1
ACS CHEMICAL BIOLOGY,4,5,5,5,5
ACS CHEMICAL NEUROSCIENCE,1,1,1,1,1
ACS NANO,2,2,2,2,2
ACTA D,5,5,5,5,5
ACTA DERMATO VENEREOLOGICA,1,1,1,1,1
ACTA DIABETOLOGICA,1,1,1,1,1
ACTA F,3,3,3,3,3
ACTA NEUROPATHOLOGICA,3,4,4,4,4


In [87]:
df1 = df['clean_journal'].value_counts()

In [88]:
df.clean_journal.unique()

array(['PSYCHOLOGICAL MEDICINE', 'BIOMACROMOLECULES',
       'JOURNAL MEDICINAL CHEMISTRY', 'J ORG CHEM',
       'JOURNAL PROTEOME RESEARCH', 'MOL PHARM', 'ACS CHEMICAL BIOLOGY',
       'JOURNAL CHEMICAL INFORMATION AND MODELING', 'BIOCHEMISTRY',
       'GASTROENTEROLOGY', 'JOURNAL BIOLOGICAL CHEMISTRY',
       'JOURNAL IMMUNOLOGY', 'ACS CHEMICAL NEUROSCIENCE', 'ACS NANO',
       'AMERICAN CHEMICAL SOCIETY', 'ANALYTICAL CHEMISTRY',
       'BIOCONJUGATE CHEMISTRY', 'JOURNAL AMERICAN CHEMICAL SOCIETY',
       'CHEST', 'JOURNAL NEUROPHYSIOLOGY', 'JOURNAL PHYSIOLOGY',
       'AMERICAN JOURNAL PSYCHIATRY', 'BEHAVIORAL NEUROSCIENCE', 'EMOTION',
       'HEALTH PSYCHOLOGY', 'JOURNAL ABNORMAL PSYCHOLOGY',
       'JOURNAL CONSULTING AND CLINICAL PSYCHOLOGY',
       'JOURNAL EXPERIMENTAL PSYCHOLOGY ANIMAL BEHAVIOR PROCESS',
       'JOURNAL EXPERIMENTAL PSYCHOLOGY HUMAN PERCEPTION AND PERFORMANCE',
       'JOURNAL FAMILY PSYCHOLOGY', 'PSYCHOLOGICAL ASSESSMENT',
       'PSYCHOLOGICAL REVIEW',
     

In [89]:
df.describe()

,ID,Publisher,Journal,Article,Cost,clean_journal
count,1928,2126,2126,2126,2126,2126
unique,1880,298,984,2125,1401,779
top,-,Elsevier,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£2040.00,PLOS ONE
freq,7,387,92,2,94,213


In [90]:
df1 = df.groupby('clean_journal').count()
df1

,ID,Publisher,Journal,Article,Cost
clean_journal,,,,,
ACADEMY NUTRITION AND DIETETICS,1,1,1,1,1
ACS CHEMICAL BIOLOGY,4,5,5,5,5
ACS CHEMICAL NEUROSCIENCE,1,1,1,1,1
ACS NANO,2,2,2,2,2
ACTA D,5,5,5,5,5
ACTA DERMATO VENEREOLOGICA,1,1,1,1,1
ACTA DIABETOLOGICA,1,1,1,1,1
ACTA F,3,3,3,3,3
ACTA NEUROPATHOLOGICA,3,4,4,4,4


In [91]:
df['clean_journal'].value_counts().head(5)

PLOS ONE                                 213
JOURNAL BIOLOGICAL CHEMISTRY              65
PROCEEDINGS NATIONAL ACADEMY SCIENCES     37
NEUROIMAGE                                31
NUCLEIC ACIDS RESEARCH                    29
Name: clean_journal, dtype: int64

These are the five most common journals. I will now calculate the total articles for each.

In [92]:
df1.sort_values('Article', ascending=False).head(5)

,ID,Publisher,Journal,Article,Cost
clean_journal,,,,,
PLOS ONE,203,213,213,213,213
JOURNAL BIOLOGICAL CHEMISTRY,64,65,65,65,65
PROCEEDINGS NATIONAL ACADEMY SCIENCES,37,37,37,37,37
NEUROIMAGE,30,31,31,31,31
NUCLEIC ACIDS RESEARCH,28,29,29,29,29


These are the five most common journals and their total articles. Next, I will calculate the mean, median and standard deviation of the open-access cost per article for each journal.

In [94]:
df['Cost'] = df['Cost'].str.lstrip('£').str.rstrip('$').astype(float)

These are the five most common journals and their total articles. Next, I will calculate the mean, median and standard deviation of the open-access cost per article for each journal.

In [95]:
df.Cost.describe()

count      2126.000000
mean      24072.451609
std      146895.027687
min           0.000000
25%        1280.000000
50%        1883.860000
75%        2321.230000
max      999999.000000
Name: Cost, dtype: float64

The mean, median and standard deviation for all cost per articles are 24072.45, 1883.86, and 146895.03 respectively. Next, I will calcualate the mean, median and standard deviation for each of our top five journals.

In [96]:
plos = df.loc[df['clean_journal'] == 'PLOS ONE']
j_biochem = df.loc[df['clean_journal'] == 'JOURNAL BIOLOGICAL CHEMISTRY']
proc_nas = df.loc[df['clean_journal'] == 'PROCEEDINGS NATIONAL ACADEMY SCIENCES']
neuroimage = df.loc[df['clean_journal'] == 'NEUROIMAGE']
nuc_acids = df.loc[df['clean_journal'] == 'NUCLEIC ACIDS RESEARCH']

In [97]:
plos.Cost.describe()

count       213.000000
mean      39361.418357
std      190670.090657
min         122.310000
25%         852.950000
50%         902.120000
75%        1044.740000
max      999999.000000
Name: Cost, dtype: float64

In [98]:
j_biochem.Cost.describe()

count        65.000000
mean      32106.840154
std      173795.906174
min         265.670000
25%        1150.600000
50%        1311.730000
75%        1600.250000
max      999999.000000
Name: Cost, dtype: float64

In [99]:
proc_nas.Cost.describe()

count        37.000000
mean      27779.029459
std      164272.161311
min         206.320000
25%         625.000000
50%         732.250000
75%         793.020000
max      999999.000000
Name: Cost, dtype: float64

In [100]:
neuroimage.Cost.describe()

count      31.000000
mean     2212.181290
std       273.193244
min      1747.160000
25%      2030.895000
50%      2326.430000
75%      2408.680000
max      2518.230000
Name: Cost, dtype: float64

In [101]:
nuc_acids.Cost.describe()

count      29.000000
mean     1162.344828
std       442.150934
min       710.000000
25%       852.000000
50%       852.000000
75%      1704.000000
max      2184.000000
Name: Cost, dtype: float64